# Reweight GPT

Reweight-GPT: An alternative to self-attetnion. 

This script is directly adapted from Andrej Karpathy's GPT project for easy comparsion.

The self-attention parts are replaced with direct re-weighting mechanism

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [29]:
text = """

"""

In [ ]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
    # text = f.read()

In [3]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

hidden_nodes = 20 # this parameter controls the number of the middle nodes for self.wr
# ------------

In [4]:
torch.manual_seed(1337)

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
chars

['\n',
 ' ',
 '-',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '9',
 ':',
 '[',
 ']',
 '。',
 '一',
 '七',
 '万',
 '三',
 '上',
 '下',
 '不',
 '专',
 '且',
 '世',
 '业',
 '东',
 '个',
 '中',
 '丸',
 '为',
 '么',
 '之',
 '乐',
 '也',
 '买',
 '了',
 '事',
 '二',
 '于',
 '五',
 '些',
 '交',
 '产',
 '人',
 '什',
 '仅',
 '今',
 '介',
 '从',
 '仓',
 '他',
 '以',
 '们',
 '件',
 '价',
 '任',
 '份',
 '会',
 '传',
 '伦',
 '伸',
 '伺',
 '但',
 '位',
 '低',
 '体',
 '何',
 '作',
 '你',
 '佣',
 '俗',
 '保',
 '信',
 '倒',
 '候',
 '借',
 '值',
 '假',
 '做',
 '像',
 '儿',
 '充',
 '先',
 '党',
 '兜',
 '入',
 '全',
 '八',
 '六',
 '关',
 '其',
 '具',
 '内',
 '再',
 '写',
 '冲',
 '净',
 '凑',
 '几',
 '出',
 '分',
 '切',
 '列',
 '则',
 '创',
 '初',
 '利',
 '别',
 '到',
 '制',
 '刺',
 '刻',
 '前',
 '力',
 '功',
 '加',
 '动',
 '势',
 '包',
 '区',
 '十',
 '千',
 '升',
 '半',
 '单',
 '卖',
 '历',
 '原',
 '去',
 '又',
 '及',
 '反',
 '发',
 '取',
 '句',
 '另',
 '只',
 '可',
 '台',
 '史',
 '号',
 '吃',
 '各',
 '合',
 '同',
 '后',
 '吧',
 '听',
 '呀',
 '告',
 '呢',
 '和',
 '品',
 '哪',
 '哭',
 '售',
 '唯',
 '商',
 '嘛',
 '回',
 '因',
 '国',
 '图',
 '圈',
 '在',
 '地

In [7]:
encode(text)

[0,
 494,
 352,
 49,
 312,
 21,
 304,
 79,
 521,
 496,
 523,
 419,
 304,
 154,
 390,
 79,
 521,
 496,
 524,
 0,
 0,
 128,
 105,
 1,
 386,
 317,
 435,
 519,
 1,
 440,
 240,
 260,
 470,
 6,
 1,
 1,
 5,
 3,
 5,
 6,
 2,
 3,
 10,
 2,
 5,
 8,
 1,
 4,
 4,
 11,
 6,
 3,
 1,
 133,
 432,
 39,
 1,
 341,
 335,
 0,
 0,
 251,
 53,
 308,
 27,
 449,
 418,
 523,
 478,
 409,
 503,
 36,
 40,
 89,
 323,
 523,
 503,
 521,
 496,
 14,
 0,
 51,
 30,
 45,
 31,
 503,
 521,
 496,
 151,
 524,
 161,
 30,
 306,
 474,
 476,
 327,
 301,
 504,
 236,
 177,
 425,
 194,
 66,
 165,
 420,
 521,
 496,
 523,
 385,
 102,
 166,
 448,
 523,
 304,
 420,
 44,
 331,
 213,
 521,
 496,
 345,
 118,
 471,
 364,
 14,
 0,
 251,
 378,
 36,
 20,
 146,
 139,
 290,
 279,
 523,
 476,
 21,
 304,
 27,
 215,
 449,
 370,
 252,
 523,
 91,
 201,
 308,
 301,
 75,
 242,
 31,
 446,
 448,
 151,
 523,
 69,
 308,
 115,
 182,
 328,
 181,
 503,
 88,
 160,
 523,
 21,
 188,
 20,
 115,
 182,
 129,
 261,
 127,
 140,
 292,
 396,
 449,
 15,
 449,
 14,
 0,
 188,


In [8]:
data

tensor([  0, 494, 352,  ...,  36,  14,   0])

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class Head(nn.Module):
    """ self-attention  parts are commented. If you want to compare them with the direct reweighting uncomment them and remove wei=self.wr(x)"""

    def __init__(self, head_size):
        super().__init__()
        
        # self.key = nn.Linear(n_embd, head_size, bias=False)
        # self.query = nn.Linear(n_embd, head_size, bias=False)

        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.wr = nn.Sequential( nn.Linear(n_embd, hidden_nodes), nn.ReLU(), nn.Linear(hidden_nodes, block_size),) # this can be a linear layer but added non-linearity with some hidden_nodes for finer control of param number
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape

        # k = self.key(x)   # (B,T,hs)
        # q = self.query(x) # (B,T,hs)
        # wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        wei = self.wr(x)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [13]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [14]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [15]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [16]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.267853 M parameters


In [17]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 6.2752, val loss 6.2847
step 500: train loss 0.3153, val loss 6.7114
step 1000: train loss 0.1748, val loss 7.5776
step 1500: train loss 0.1514, val loss 8.0265
step 2000: train loss 0.1390, val loss 8.3342
step 2500: train loss 0.1358, val loss 8.7169
step 3000: train loss 0.1203, val loss 8.6746
step 3500: train loss 0.1174, val loss 9.0160
step 4000: train loss 0.1193, val loss 9.0930
step 4500: train loss 0.1182, val loss 9.3440
step 4999: train loss 0.1135, val loss 9.4117


In [25]:
# generate from the model
context = torch.zeros((1, block_size), dtype=torch.long, device=device)
context += xb[0, :]
gen_text = decode(m.generate(context, max_new_tokens=3000)[0].tolist())
# gen_text
print(gen_text)